In [1]:
import os
import csv
import spacy
from collections import defaultdict
from spacy.tokens import DocBin

In [2]:
# 基本設定
main_directory = "processed_data"  # 親ディレクトリへの相対パス
subdirectories = ["economist"]  # 読み込む子ディレクトリ名のリスト
nlp_model = "en_core_web_sm"  # 解析に利用したモデル名

# 特定のファイルのみを読み込む場合は以下のリストにtxtファイル名を追加(空リストの場合は全ファイルを読み込む)
selected_file_names = []

In [3]:
# spaCyの言語モデルをロード
nlp = spacy.load(nlp_model)

# 解析結果を格納する辞書の初期化
docs_dict = {}
# フィルタリングされたDocオブジェクトを格納する辞書の初期化
filtered_docs = {}

# 指定されたサブディレクトリ内のファイルを読み込む
for subdir in subdirectories:
    directory = os.path.join(main_directory, subdir)
    
    # サブディレクトリ内のファイルを読み込む
    for filename in os.listdir(directory): 
        if filename.endswith(".spacy"):  # .spacyファイルならば
            spacy_path = os.path.join(directory, filename)
            txt_filename = filename.replace(".spacy", ".txt")  # .spaCyファイルに対応するテキストファイル名を作成
            txt_path = os.path.join(directory, txt_filename)  # .spacyファイルに対応するテキストファイルのパス
            
            # .spacyファイルからDocオブジェクトを読み込む
            doc_bin = DocBin().from_disk(spacy_path)  # DocBinオブジェクトを読み込む
            docs = list(doc_bin.get_docs(nlp.vocab))  # Docオブジェクトをリストに格納
            
            # 対応する.txtファイルからファイル名を読み込む
            if os.path.exists(txt_path):
                with open(txt_path, 'r', encoding='utf-8') as f:
                    file_names = [line.strip() for line in f.readlines()]  # ファイル名をリストに格納
                
                # Docオブジェクトとファイル名の対応を辞書に格納
                for doc, fname in zip(docs, file_names):
                    docs_dict[fname] = doc
            else:
                print(f"警告: {txt_filename} に対応するテキストファイルが存在しません。")

# ユーザーがファイル名を指定した場合、それに対応するDocオブジェクトをフィルタリング
if selected_file_names:
    missing_files = []  # 存在しないファイル名を格納するリスト
    for fname in selected_file_names:
        if fname in docs_dict:  # 指定されたファイル名が読み込んだデータに存在する場合
            filtered_docs[fname] = docs_dict[fname]
        else:
            missing_files.append(fname)
    
    if missing_files:
        print(f"警告: 次の指定されたファイルは存在しません: {', '.join(missing_files)}")
    if filtered_docs:
        print("指定された一部のデータが読み込まれました。")
    else:
        print("指定されたファイルに対応するデータが見つかりませんでした。")
else:
    filtered_docs = docs_dict
    print("全てのデータが読み込まれました。")
print(f"読み込まれたデータ数: {len(filtered_docs)}")
print(f"読み込まれたファイル名: {list(filtered_docs.keys())}")

全てのデータが読み込まれました。
読み込まれたデータ数: 1
読み込まれたファイル名: ['cleaned.txt']


# モデルに付属するパイプライン機能

In [ ]:
# モデルに付属するパイプライン機能を表示する
print("モデルに付属するパイプライン機能:")
for pipe_name in nlp.pipe_names:
    print(f"- {pipe_name}")

# モデルに付属するストップワード

In [5]:
# モデルに付属するデフォルトのストップワードリストをテキストファイルに出力する
stopwords_filename = f"{nlp_model}_stopwords.txt"
with open(stopwords_filename, "w", encoding="utf-8") as f:
    for stopword in sorted(nlp.Defaults.stop_words):
        f.write(f"{stopword}\n")
print(f"モデルに付属するデフォルトのストップワードリストが {stopwords_filename} に出力されました。")

モデルに付属するデフォルトのストップワードリストが en_core_web_sm_stopwords.txt に出力されました。


# 品詞, 品詞タグ, 依存関係ラベル, 固有表現ラベルの出力

In [4]:
import csv
from collections import Counter

# 品詞、品詞タグ、依存構造ラベル、固有表現ラベルを格納する辞書の初期化
pos_counts = Counter()
tag_counts = Counter()
dep_counts = Counter()
ent_counts = Counter()

# トークン数と固有表現数の初期化
total_tokens = 0
total_ents = 0

# フィルタリングされたDocオブジェクトを反復処理
for doc in filtered_docs.values():
    # 品詞の集計
    for token in doc:
        pos_counts[token.pos_] += 1
        tag_counts[token.tag_] += 1
        dep_counts[token.dep_] += 1
        total_tokens += 1
    
    # 固有表現ラベルの集計
    for ent in doc.ents:
        ent_counts[ent.label_] += 1
        total_ents += 1

# CSVファイルに結果を書き込む
with open('output.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    
    # ヘッダー行の書き込み
    header = ['品詞ラベル', '品詞頻度', '品詞割合', '品詞説明',
              '品詞タグラベル', '品詞タグ頻度', '品詞タグ割合', '品詞タグ説明',
              '依存構造ラベル', '依存構造頻度', '依存構造割合', '依存構造説明',
              '固有表現ラベル', '固有表現頻度', '固有表現割合', '固有表現説明']
    writer.writerow(header)
    
    # データ行の書き込み
    max_rows = max(len(pos_counts), len(tag_counts), len(dep_counts), len(ent_counts))
    for i in range(max_rows):
        row = []
        
        # 品詞情報の追加
        if i < len(pos_counts):
            pos, count = list(pos_counts.items())[i]
            percentage = count / total_tokens * 100
            row.extend([pos, count, f"{percentage:.2f}%", spacy.explain(pos)])
        else:
            row.extend(['', '', '', ''])
        
        # 品詞タグ情報の追加
        if i < len(tag_counts):
            tag, count = list(tag_counts.items())[i]
            percentage = count / total_tokens * 100
            row.extend([tag, count, f"{percentage:.2f}%", spacy.explain(tag)])
        else:
            row.extend(['', '', '', ''])
        
        # 依存構造情報の追加
        if i < len(dep_counts):
            dep, count = list(dep_counts.items())[i]
            percentage = count / total_tokens * 100
            row.extend([dep, count, f"{percentage:.2f}%", spacy.explain(dep)])
        else:
            row.extend(['', '', '', ''])
        
        # 固有表現情報の追加
        if i < len(ent_counts):
            ent_label, count = list(ent_counts.items())[i]
            percentage = count / total_ents * 100
            row.extend([ent_label, count, f"{percentage:.2f}%", spacy.explain(ent_label)])
        else:
            row.extend(['', '', '', ''])
        
        writer.writerow(row)

print("結果がCSVファイル 'output.csv' に書き込まれました。")

結果がCSVファイル 'output.csv' に書き込まれました。


c:\Users\G018C1286\zemi\zemi2\kasou\Lib\site-packages\spacy\glossary.py:20: UserWarning: [W118] Term 'predet' not found in glossary. It may however be explained in documentation for the corpora used to train the language. Please check `nlp.meta["sources"]` for any relevant links.
  warnings.warn(Warnings.W118.format(term=term))


# 総語数, 異なり語数, 文の数(is alphaは日本語とかに対応していないので修正必要！)

In [14]:
import spacy
from collections import Counter

def calculate_corpus_stats(docs):
    total_tokens = 0
    token_counts = Counter()
    total_sentences = 0

    # 除外する品詞のリスト
    excluded_pos = {"PUNCT", "SYM", "X", "NUM"}

    for doc in docs:
        # 記号、句読点、その他のカテゴリ、数値を除外
        tokens = [token.lemma_ for token in doc if token.pos_ not in excluded_pos]
        token_counts.update(tokens)
        total_tokens += len(tokens)
        total_sentences += len(list(doc.sents))
    
    # 計算結果の表示
    print(f"総語数: {total_tokens}")
    print(f"異なり語数: {len(token_counts)}")
    print(f"文の数: {total_sentences}")
    
    # 5回以上出現した単語数
    words_above_5 = sum(1 for word, count in token_counts.items() if count >= 5)
    print(f"\n5回以上出現した単語数: {words_above_5}")

    # 10回以上出現した単語数
    words_above_10 = sum(1 for word, count in token_counts.items() if count >= 10)
    print(f"10回以上出現した単語数: {words_above_10}")

# 関数を呼び出して結果を表示
calculate_corpus_stats(filtered_docs.values())

総語数: 122515
異なり語数: 9331
文の数: 6733

5回以上出現した単語数: 2376
10回以上出現した単語数: 1396


In [13]:

import spacy
from collections import Counter

def calculate_corpus_stats(docs):
    total_tokens = 0
    token_counts = Counter()
    total_sentences = 0

    # 除外する品詞のリスト
    excluded_pos = {"PUNCT", "SYM", "X", "NUM"}

    for doc in docs:
        # 記号、句読点、その他のカテゴリ、数値を除外
        tokens = [token.lemma_ for token in doc if token.pos_ not in excluded_pos and not token.like_num and token.is_alpha]
        token_counts.update(tokens)
        total_tokens += len(tokens)
        total_sentences += len(list(doc.sents))
    
    # 計算結果の表示
    print(f"総語数: {total_tokens}")
    print(f"異なり語数: {len(token_counts)}")
    print(f"文の数: {total_sentences}")
    
    # 5回以上出現した単語数
    words_above_5 = sum(1 for word, count in token_counts.items() if count >= 5)
    print(f"\n5回以上出現した単語数: {words_above_5}")

    # 10回以上出現した単語数
    words_above_10 = sum(1 for word, count in token_counts.items() if count >= 10)
    print(f"10回以上出現した単語数: {words_above_10}")

# 関数を呼び出して結果を表示
calculate_corpus_stats(filtered_docs.values())

総語数: 119349
異なり語数: 9013
文の数: 6733

5回以上出現した単語数: 2341
10回以上出現した単語数: 1379


# 文の境界認識機能の確認

In [ ]:
# 抽出するサンプル文の数と出力ファイル名を指定
num_sentences = 10
output_file = "sample_sentences.txt"

In [36]:
def save_sample_sentences(docs, output_file, num_sentences):
    sentences = []
    for doc in docs.values():
        for sent in doc.sents:
            sentences.append(sent.text.strip())
            if len(sentences) >= num_sentences:
                break
        if len(sentences) >= num_sentences:
            break

    with open(output_file, "w", encoding="utf-8") as f:
        f.write("\n".join(sentences))

    print(f"指定された {num_sentences} 個の文が {output_file} に出力されました。")

save_sample_sentences(filtered_docs, output_file, num_sentences)

指定された 10 個の文が sample_sentences.txt に出力されました。
